# Kaggle Challenge

The objective of this challenge was to classify 102 different insects present on approximately 23000 test images with about 53000 training data at our disposal. I decided to start with transfer learning and I relied on these different keras documentations for that:

https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/  
https://keras.io/api/applications/  
https://keras.io/api/applications/#usage-examples-for-image-classification-models

Knowing that the number of data was limited and the training time too, I wanted a pre-trained model with a reasonable number of parameters and a good accuracy. I wanted to turn first to efficientNet algorithms having very good results with few parameters but during the training of the model there was a recurrent error on the efficientNet that I did not have on the other models so I could not use them (the training work but in a very inefficient way). After analysis and testing on other models I decided to use the InceptionV3.
During this challenge,the main problem was to manage the class imbalance by using the f1-score metrics and taking into account the weight in classes without this the f1-score we were evaluated on was very bad.
And the difficulty that restricted me the most was not so much the 30 hours of GPU available but rather the 12 hours maximum of execution of a kaggle notebook. Indeed, I had the surprise that my training was not finished and that nothing could be saved and that I lost 12 hours. I am now much more aware of the time needed for each task, I also learned to save the weights of my models so that I don't lose them and have to start the training again each time.

## Importing data

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import glob

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TensorFlow")

For the importation of the data, being given that they were not all of the same size and that the labels were stored in a separate csv file I left on an imageDataGenerator. This one allows of course not to have to load all the images at once and saturate the ram, to preprocess the images as one wishes it (in particular the adaptation of the size images without cropping them not to lose information) and allows to make data increase by the same occasion thus that corresponded completely to what I wished. For the preprocessing of the data a function dedicated to the Inception model exists in Keras, for the data augmentation I used rotation/translation/version of the images and enlargement/reduction of them.
I decide to use a small batch size number because of the complexity of the data set, so i took 32.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
#Setting important parameters
image_size = 299 #Size required for the InceptionV3 model
batch_size = 32

In [ ]:
# Load labels from CSV file
df = pd.read_csv('/kaggle/input/polytechnice2023-deep-learning-competition/train.csv')
df['y'] = df['y'].astype(str)

# Define data transformations
# Preprocessing + Data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,#Preprocessing images for InceptionV3
    rotation_range=20,# Rotate images
    width_shift_range=0.2,# Shifting the image to the sides
    height_shift_range=0.2,# Same upwards
    zoom_range=0.2,# Enlargement/Reduction of the image
    horizontal_flip=True,# Horizontal image inversion
    vertical_flip=True,# Same Vertical
    validation_split=0.2 # Separation of data in validation and train
)

# Pre-processing of training data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,# The dataframe where the labels are stored
    #The directory where the images are stored
    directory='/kaggle/input/polytechnice2023-deep-learning-competition/train_images',
    x_col='ID',# Where the image names are stored
    y_col= 'y',# Where the labels are stored
    target_size=(image_size,image_size),#Adapting the image to the required size
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Using training data
)

# Pre-processing of validation data
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory='/kaggle/input/polytechnice2023-deep-learning-competition/train_images',
    x_col='ID',
    y_col='y',
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use validation data
)

In [ ]:
class_weights = compute_class_weight(class_weight = 'balanced',
                                     classes = np.unique(train_generator.classes),
                                     y = train_generator.classes)
class_weight_dict = dict(zip(np.unique(train_generator.classes), class_weights))
print(class_weight_dict)

This is the dictionary to take into account the weight of the classes.

In [ ]:
import matplotlib.pyplot as plt
# Images and labels for the next batch
images, labels = next(train_generator)

# Display the first image
plt.imshow(images[0])
plt.title('Label: {}'.format(labels[0]))
plt.show()

## Model

The choice of the number of epochs was imposed on me by the 12 hours of the kaggle so I decided to do 15 for the training but in the end I only used 10 because I noticed that we quickly reach the maximum precision by training the last layers, certainly because the model has not been trained on insect recognition and our dataset is complicated, that's why I finally put more epoch for the fine tuning.

In [ ]:
# Number of classes and epochs
epochs = 10
classes = 102

Definition of the model with a GlobalAveragePooling2D layer to reduce x to a 1D vector, BatchNormalization and dropout to facilitate learning (as for the dropout rate I didn't really have the time to try several so I took the one from the keras doc) and finally 2 Dense layers with one fully connected and one prediction layer for the 102 classes
First I freeze the pre-trainer model so that the prediction layers can be trained in priority.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout,BatchNormalization

with tf.device('/device:GPU:0'):
    # Loading InceptionV3 without the last layers of classifications
    # weights='imagenet' to load the weights
    base_model = InceptionV3(weights='imagenet', include_top=False)
    # Adding the classification layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    top_dropout_rate = 0.2
    x = Dropout(top_dropout_rate, name="top_dropout")(x)
    x = Dense(1280, activation='relu')(x)
    predictions = Dense(classes, activation='softmax')(x)

    # New model
    model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Freeze of pre-trained model layers
for layer in base_model.layers:
    layer.trainable = False

## Train

For the optimizer I chose rmsprop as indicated in the keras doc and it worked very well (I tried with Adam but the difference was not very noticeable in the time I had) with a loss categorical_crossentropy.

In [ ]:
from tensorflow_addons.metrics import F1Score
f1score = F1Score(num_classes=classes, average='macro')

In [ ]:
# Compilation and training of the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=[f1score])
history = model.fit(train_generator, epochs=epochs, 
                    validation_data=valid_generator,class_weight=class_weight_dict)

In [ ]:
#To save and download the weights of my model
model.save_weights('my_model_weights.h5')
!cd /kaggle/working
from IPython.display import FileLink
FileLink(r'my_model_weights.h5')

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["f1_score"])
    plt.plot(hist.history["val_f1_score"])
    plt.title("model f1_score")
    plt.ylabel("f1_score")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(hist)

Here we see that the model trains well with a validation curve that increases as the training curve increases.

## Fine Tuning

For the fine tuning I did not unfreeze the whole model as indicated in the keras doc, it seemed coherent to me to have less parameters to train given our quantity of images at disposal.

In [ ]:
# Defreeze certaines couche du model préentrainé
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

I decided to use early stopping to prevent overfitting but this was not necessary as I could not train my model long enough for this.

In [ ]:
#Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
ourCallback = EarlyStopping(monitor='f1_score', min_delta=0.0001, patience=20, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

I try many optimizer for fine tuning like SGD(lr=0.0001, momentum=0.9) in the keras documentation but my accuracy/f1 score fall drastically. I try RMSprop just with a really low step and that was great i have a good fine tuning.

In [ ]:
# Compilation and training
from tensorflow.keras.optimizers import RMSprop
epochs = 15

model.compile(optimizer=RMSprop(learning_rate=0.00001), loss='categorical_crossentropy', metrics=[f1score])
history = model.fit(train_generator, epochs=epochs, 
                    validation_data=valid_generator,callbacks=[ourCallback],verbose=1,class_weight=class_weight_dict)

In [ ]:
#To save and download the weights of my model
model.save_weights('my_model_weights.h5')
!cd /kaggle/working
from IPython.display import FileLink
FileLink(r'my_model_weights.h5')

I had originally put another train phase by freezing the pre-trainer model I thought it could have been useful but no it was a bad idea I lost accuracy so I decided to remove it.

## Test

For the test data I had to batch load it by hand because with an image generator and flow_from_directory it saturated my RAM.
Then I simply predicted and stored the data in a dataframe that I converted to csv.

In [ ]:
#Load weights of my model
model.load_weights('/kaggle/input/weigths/my_model_weights (4).h5')

In [ ]:
#Loading test data
import os
from tensorflow.keras.preprocessing import image

# path of the directory where the images are
image_dir = '/kaggle/input/polytechnice2023-deep-learning-competition/test_images'
# All paths of all images
test_images = ["/kaggle/input/polytechnice2023-deep-learning-competition/train_images/00006.jpg",
               "/kaggle/input/polytechnice2023-deep-learning-competition/train_images/00007.jpg",
              "/kaggle/input/polytechnice2023-deep-learning-competition/train_images/00008.jpg",
              "/kaggle/input/polytechnice2023-deep-learning-competition/train_images/00009.jpg",
              "/kaggle/input/polytechnice2023-deep-learning-competition/train_images/00010.jpg"]
#test_images = glob.glob(image_dir + '/*.jpg')

def test_data_generator(batch_size):
    i = 0
    while i < len(test_images):
        batch_images = test_images[i:i+batch_size] #batch images
        #Array empty for batch images a preprocess
        batch_data = np.zeros((len(batch_images), image_size, image_size, 3))
        #Batch image path list
        batch_filenames = []
        for j, img_path in enumerate(batch_images):
            img = image.load_img(img_path, target_size=(image_size, image_size))
            x = image.img_to_array(img)
            x = preprocess_input(x)#Preprocessing of the image
            batch_data[j] = x
            batch_filenames.append(os.path.basename(img_path))
        i += batch_size
        yield batch_data,batch_filenames

batch_size = 256
test_data = test_data_generator(batch_size)

# Label prediction
predictions_list = []
filename_list = []
for batch_data,batch_filenames in test_data:
    predictions = model.predict(batch_data)
    predictions_cat = np.argmax(predictions, axis=1)#Converting labels to int
    print(predictions_cat)
    predictions_list.extend(predictions_cat)
    filename_list.extend(batch_filenames)
    print(filename_list)


In [ ]:
# Creation of a dataFrame with predictions
results = pd.DataFrame({
    "ID": filename_list,
    "Prediction": predictions_list,
})
# Convert it to csv
results.to_csv("test_results.csv", index=False)

In [ ]:
# To download it
!cd /kaggle/working
from IPython.display import FileLink
FileLink(r'test_results.csv')

After Monday's session I tried with images from the train dataset and the predictions are not good so the error is not in the conversion of my results to csv. I spent a lot of time on it but I can't see where my error is which is a bit of a shame because my training and validation results are relatively good.